# Capítulo 4 — Aprendizado Supervisionado (Classificação)

Modelos: Regressão Logística, KNN, Árvore de Decisão.


In [ ]:
from sklearn.datasets import load_iris
from sklearn.metrics import classification_report
from src.preprocessing import train_test_scale
from src.models import make_classifiers

data = load_iris(as_frame=True)
X = data.frame.drop(columns=["target"])
y = data.frame["target"]

X_train, X_test, y_train, y_test, _ = train_test_scale(X, y)

models = make_classifiers()
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"\n[{name}]")
    print(classification_report(y_test, y_pred))
